In [2]:

import torch 
from torchvision import transforms,datasets
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import SubsetRandomSampler
import matplotlib.pyplot as plt 
import numpy as np 


c:\Users\ASUS\.conda\envs\torch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset gifted by torchvision

In [3]:

transform=transforms.Compose([transforms.ToTensor()])
# To get the Normalization values do the follwing after downloading train data
# print(train_data.data.float().mean()/255)
# print(train_data.data.float().std()/255)

In [4]:

train_data=datasets.FashionMNIST('data',train=True,download=True,transform=transform)
test_data=datasets.FashionMNIST('data',train=False,download=True,transform=transform)

100%|██████████| 26421880/26421880 [04:05<00:00, 107428.78it/s]


Extracting data\FashionMNIST\raw\train-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 29515/29515 [00:00<00:00, 165622.52it/s]


Extracting data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 4422102/4422102 [00:08<00:00, 491887.56it/s] 


Extracting data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to data\FashionMNIST\raw



100%|██████████| 5148/5148 [00:00<?, ?it/s]

Extracting data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to data\FashionMNIST\raw



In [5]:
valid_size=0.2
train_length=len(train_data)

indices=[i for i in range(train_length)]
np.random.shuffle(indices)

split=int(np.floor(valid_size*train_length))
train_idx=indices[split:]
valid_idx=indices[:split]

In [8]:
num_workers=0
batch_size=20
train_loader=torch.utils.data.DataLoader(train_data,batch_size=batch_size,num_workers=num_workers)
valid_loader=torch.utils.data.DataLoader(train_data,batch_size=batch_size,num_workers=num_workers)
test_loader=torch.utils.data.DataLoader(test_data,batch_size=batch_size,num_workers=num_workers)

# This is for debugging
print(f"Training data size : {train_idx.__len__()}, Validation data size : {valid_idx.__len__()}, Test data size : {test_loader.dataset.__len__()}")
     

Training data size : 48000, Validation data size : 12000, Test data size : 10000


Then we will do a single project. We will create a dataset for image denoising. We will create training, validation and test dataset with by adding some noises on the clean images. 


In [ ]:
from shutil import copy2
import os 
import cv2
import glob
import platform
delimeter = "\\" if platform.system() == "Windows" else "/"

In [ ]:
def create_folder(data_dir, target_dir):
    target = os.path.join(data_dir, target_dir)
    if not os.path.exists(target):
        os.mkdir(target)
    return target
                                    
def writefile(target_dir, filename, data):
    filepath = os.path.join(target_dir, filename)
    cv2.imwrite(filepath, data)
    
                                                 
def copyfile(source_dir, dest_dir):
    sources = glob.glob(os.path.join(source_dir,'*'))
    for i in sources:
        filename = i.split('\\')[-1]
        filepath = os.path.join(dest_dir, filename)
        copy2(i, filepath)

def add_noise(input_img,sigma):
    noise = np.random.normal(scale=sigma/255, size=input_img.shape)
    add_noise = np.clip(input_img + noise, 0, 1)
    return np.float32(add_noise)

In [7]:
def prepare_training_dataset(dataset_train_dir, dataset_val_dir, save_dir):
    clean_train = glob.glob(os.path.join(dataset_train_dir,'*'))
    clean_train.sort()
    
    noisy_input_dir = create_folder(os.path.join(save_dir,"train_trial"),"input")
    clean_target_dir = create_folder(os.path.join(save_dir,"train_trial"), "gt")
    copyfile(dataset_train_dir, clean_target_dir)
    for i in clean_train:
        im_path = i
        im_name = im_path.split(delimeter)[-1]
        img = cv2.imread(i)
        y1 = np.uint8(255*add_noise(img/255, sigma=50))
        cv2.imwrite(os.path.join(noisy_input_dir,im_name), y1)
    print("training noisy data prepared")
    
    clean_val = glob.glob(os.path.join(dataset_val_dir,'*'))
    clean_val.sort()
    noisy_val_dir = create_folder(os.path.join(save_dir,"val_trial"), "noisy")
    clean_val_dir = create_folder(os.path.join(save_dir,"val_trial"), "clean")
    copyfile(dataset_val_dir, clean_val_dir)
    for k in clean_val:
        val_path = k
        val_name = val_path.split(delimeter)[-1]
        img_val = cv2.imread(k)
        y_val = np.uint8(255*add_noise(img/255, sigma=50))
        target_dir_val = os.path.join(noisy_val_dir, val_name)
        cv2.imwrite(target_dir_val, y_val)
    print("validation noisy data prepared")

In [ ]:
print("preparing training and validation data................!!!!")
dataset_train_dir = 'Give your path here' 
dataset_val_dir = 'Give your path here'
save_dir = 'Give your path here'
prepare_training_dataset(dataset_train_dir, dataset_val_dir, save_dir)

In [ ]:
def prepare_testdataset(data_dir, dataset_name):
    files = glob.glob(os.path.join(data_dir, dataset_name, '*.png'))
    files.sort()
    noisy_dir = create_folder(data_dir, dataset_name + "_noisy")
    for i in range(len(files)):
        im_path = i
        im_name = im_path.split('\\\\')[-1]
        img = cv2.imread(files[i])
        test = np.uint8(255*add_noise(img/255, sigma=50))
        cv2.imwrite(os.path.join(noisy_dir,im_name), test)
    print("test noisy data prepared")

In [ ]:
data_dir = 'your directory is here'
dataset_name = 'Give your dataset name'
prepare_testdataset(data_dir, dataset_name)